In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType

# Creating the SparkSession
spark = SparkSession.builder.appName('Test').master("local").getOrCreate()

In [2]:
# Loading the data
path = "C:\\Project\\Files\\Input\\json\\Sample-JSON_1.json"
df = spark.read.option("multiline", "true").json(path)

df.show()
df.printSchema()

+-------------------+------------+-----+-----------+-------+
|               City|RecordNumber|State|ZipCodeType|Zipcode|
+-------------------+------------+-----+-----------+-------+
|PASEO COSTA DEL SUR|           1|   PR|   STANDARD|    701|
|       BDA SAN LUIS|           2|   PR|   STANDARD|    702|
|               null|           3|   PR|   STANDARD|    703|
|PASEO COSTA DEL SUR|           4|   PR|   STANDARD|    704|
|            Cuttack|           5|   PR|   STANDARD|    705|
|              Delhi|           6|   PR|   STANDARD|     12|
+-------------------+------------+-----+-----------+-------+

root
 |-- City: string (nullable = true)
 |-- RecordNumber: long (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- Zipcode: long (nullable = true)



In [3]:
# Filtering the data where the City is null

# df.where(col("City").isNotNull()).show()
without_null_df = df.filter(df.City.isNotNull()).show()

+-------------------+------------+-----+-----------+-------+
|               City|RecordNumber|State|ZipCodeType|Zipcode|
+-------------------+------------+-----+-----------+-------+
|PASEO COSTA DEL SUR|           1|   PR|   STANDARD|    701|
|       BDA SAN LUIS|           2|   PR|   STANDARD|    702|
|PASEO COSTA DEL SUR|           4|   PR|   STANDARD|    704|
|            Cuttack|           5|   PR|   STANDARD|    705|
|              Delhi|           6|   PR|   STANDARD|     12|
+-------------------+------------+-----+-----------+-------+



In [4]:
df.filter("City IS NOT NULL").count()

5

In [5]:
# Creating a Temp View from the DataFrame

df.createOrReplaceTempView("records")

sql_df = spark.sql("select Zipcode,City from(select *, dense_rank() over(order by Zipcode desc)r from records) where r=1")
print(sql_df.show(1))

+-------+-------+
|Zipcode|   City|
+-------+-------+
|    705|Cuttack|
+-------+-------+

None


In [6]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

# Creating the SparkSession
spark = SparkSession.builder.appName('Test').master("local").getOrCreate()

# Enable Arrow-based columnar data transfers
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Generate a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))
print(type(pdf))

# Create a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()
result_pdf

<class 'pandas.core.frame.DataFrame'>


,0,1,2
0,0.111739,0.231393,0.879764
1,0.201488,0.776239,0.037592
2,0.205363,0.642324,0.015205
3,0.660667,0.935614,0.815210
4,0.739469,0.327542,0.162796
...,...,...,...
95,0.566710,0.221150,0.338994
96,0.632269,0.975356,0.281749
97,0.227194,0.519037,0.614162
98,0.971867,0.673837,0.028385
